## Import libraries

In [ ]:
import os
import gc
import re
from time import time
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import torch
import transformers
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, AutoModel
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch.nn.functional as F
np.random.seed(1337)


## Tokenizer

In [ ]:
model = "/kaggle/input/llama-3/transformers/70b-chat-hf/1/llama3-70b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
tokenizer.add_eos_token = True

# save tokenizer to load offline during inference
tokenizer.save_pretrained('tokenizer')

In [ ]:
# Utility function giving token length
def get_token_lengths(texts):
    # tokenize and receive input_ids for reach text
    input_ids = tokenizer(texts.tolist(), return_tensors='np')['input_ids']
    # return length of inputs_ids for each text
    return [len(t) for t in input_ids]

## Prepare train set

In [ ]:
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
train.head(5)

In [ ]:
def put_text(train):
    train['text'] = 'User prompt: ' + train['prompt'] +  '\n\nModel A:\n' + train['response_a'] +'\n\n--------\n\nModel B:\n'  + train['response_b']
    return train

train = put_text(train)
print(train['text'][0])

In [ ]:
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

train.loc[:, 'prompt'] = train['prompt'].apply(process)
train.loc[:, 'response_a'] = train['response_a'].apply(process)
train.loc[:, 'response_b'] = train['response_b'].apply(process)


train = put_text(train)
print(train['text'][0])

In [ ]:
train.loc[:, 'token_count'] = get_token_lengths(train['text'])

# prepare label for model
train.loc[:, 'label'] = np.argmax(train[['winner_model_a','winner_model_b','winner_tie']].values, axis=1)

# Display data
display(train.head())

In [ ]:
train.label.value_counts()

In [ ]:
# token Count
display(train['token_count'].describe().to_frame().astype(int))


In [ ]:
# get length of tokens which covers 90% of data, we'll still take 1024 length!
np.percentile(train['token_count'], 90)

## Tokenize

In [ ]:
# Tokenize Data
tokens = tokenizer(
    train['text'].tolist(), 
    max_length=1024, 
    truncation=True, 
    return_tensors='np')

# Input IDs are the token IDs
INPUT_IDS = tokens['input_ids']
# Attention Masks to Ignore Padding Tokens
ATTENTION_MASKS = tokens['attention_mask']
# Label of Texts
LABELS = train[['winner_model_a','winner_model_b','winner_tie']].values

print(f'INPUT_IDS shape: {INPUT_IDS.shape}, ATTENTION_MASKS shape: {ATTENTION_MASKS.shape}')
print(f'LABELS shape: {LABELS.shape}')

In [ ]:
max_features = 14300
max_len = 1024
maxlen = max_len
batch_size = 16
embedding_dims = 100
nb_filter = 150
filter_length = 3
hidden_dims = 100
nb_epoch = 100

In [ ]:
from __future__ import print_function
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D, LSTM
from keras.datasets import imdb
from keras import backend as K
from keras.optimizers import Adadelta,Adamax
from keras.preprocessing import sequence as sq

from keras.layers import Dense, Dropout, Activation, Lambda,Input,TimeDistributed,Flatten
from keras.models import Model
from keras.callbacks import ModelCheckpoint

from tensorflow.python.keras.backend import set_session as K
num_samples = INPUT_IDS.shape[0]

# Số lượng mẫu cho X_valid (20% của X_train)
num_valid_samples = int(num_samples * 0.2)

# Xáo trộn các chỉ số của X_train
indices = np.random.permutation(num_samples)

# Chọn 20% chỉ số đầu tiên làm chỉ số cho X_valid
valid_indices = indices[:num_valid_samples]

# Các chỉ số còn lại làm chỉ số cho X_train
train_indices = indices[num_valid_samples:]

# Tạo X_valid và X_train mới từ các chỉ số đã chọn
X_train = sq.pad_sequences(INPUT_IDS[train_indices], maxlen=max_len)
X_train_attention = sq.pad_sequences(ATTENTION_MASKS[train_indices], maxlen=max_len)
y_train = LABELS[train_indices]

X_valid = sq.pad_sequences(INPUT_IDS[valid_indices], maxlen=max_len)
X_valid_attention = sq.pad_sequences(ATTENTION_MASKS[valid_indices], maxlen=max_len)
y_valid = LABELS[valid_indices]

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_valid = np.array(X_valid)
y_valid = np.array(y_valid)

## Define Model

In [ ]:
from tensorflow.keras.layers import Layer
from keras.layers import concatenate, Dropout, BatchNormalization, LSTM, Conv1D
from keras.layers import  GlobalMaxPooling1D
import tensorflow as tf

class ApplyAttentionMask(Layer):
    def call(self, inputs):
        embeddings, attention_mask = inputs
        return embeddings * tf.expand_dims(attention_mask, -1)

input_layer = Input(shape=(max_len,),dtype='int32', name='main_input')
attention_masks = Input(shape=(max_len,), dtype='float32', name="attention_masks")

emb_layer = Embedding(max_features,
                      embedding_dims,
                      input_length=max_len
                      )(input_layer)

masked_embeddings = ApplyAttentionMask(name='apply_attention_mask')([emb_layer, attention_masks])

# LSTM branch (with Batch Normalization and Dropout)
lstm_out = LSTM(128, return_sequences=True)(masked_embeddings)
lstm_out = BatchNormalization()(lstm_out) # Batch Normalization helps to normalize activations and speed up convergence
lstm_out = Dropout(0.5)(lstm_out) # Dropout = 0.5 helps to prevent overfitting
lstm_out = LSTM(64, return_sequences=True)(lstm_out)
lstm_out = BatchNormalization()(lstm_out)
lstm_out = Dropout(0.5)(lstm_out)
lstm_out = LSTM(32)(lstm_out)
lstm_out = BatchNormalization()(lstm_out)
lstm_out = Dropout(0.5)(lstm_out)

# CNN layer branch (with Batch Normalization and Dropout)
cnn_out = Conv1D(64, 5, activation='relu')(masked_embeddings)
cnn_out = BatchNormalization()(cnn_out)
cnn_out = Dropout(0.5)(cnn_out)
cnn_out = Conv1D(32, 5, activation='relu')(cnn_out)
cnn_out = BatchNormalization()(cnn_out)
cnn_out = Dropout(0.5)(cnn_out)
cnn_out = GlobalMaxPooling1D()(cnn_out)


# Concatenate LSTM and CNN outputs
merged = concatenate([lstm_out, cnn_out])
merged = Dense(32, activation='sigmoid')(merged)
merged = BatchNormalization()(merged)
merged = Dropout(0.5)(merged)
pred = Dense(3, activation='softmax')(merged)


# Build model
model = Model(inputs=[input_layer, attention_masks], outputs=[pred])
adadelta = Adadelta(learning_rate=1.0, rho=0.75, epsilon=1e-06)
adamax = Adamax(learning_rate=0.001)
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## Training

In [ ]:
def clip_indices(data, max_index):
    return np.where(data >= max_index, max_index - 1, data)
X_train = clip_indices(X_train, 14300)
X_train_attention = clip_indices(X_train_attention, 14300)
X_valid = clip_indices(X_valid, 14300)
X_valid_attention = clip_indices(X_valid_attention, 14300)

In [ ]:
from keras.callbacks import EarlyStopping
checkpoint = ModelCheckpoint('/kaggle/working/model.keras',
                                 monitor='val_acc', verbose=0, save_best_only=True,
                                 mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

model.fit([X_train,X_train_attention], y_train,
          batch_size=16,
          epochs=nb_epoch,
#           callbacks=[checkpoint, early_stopping],
          callbacks=[early_stopping],
          validation_data=([X_valid,X_valid_attention], y_valid))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.save('model.keras', overwrite=True)
model.save_weights("model.weights.h5", overwrite=True)

## Test Model

In [ ]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')


test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

# Drop 'Null' for training
indexes = test[(test.response_a == 'null') & (test.response_b == 'null')].index
test.drop(indexes, inplace=True)
test.reset_index(inplace=True, drop=True)

print(f"Total {len(indexes)} Null response rows dropped")
print('Total train samples: ', len(test))

In [ ]:
test.head()

In [ ]:
test['text'] = 'User prompt: ' + test['prompt'] +  '\n\nModel A:\n' + test['response_a'] +'\n\n--------\n\nModel B:\n'  + train['response_b']
print(test['text'])

In [ ]:
# Tokenize Data
tokens_test = tokenizer(
    test['text'].tolist(), 
    max_length=1024, 
    truncation=True, 
    return_tensors='np')

# Input IDs are the token IDs
INPUT_test = tokens_test['input_ids']
# Attention Masks to Ignore Padding Tokens
ATTENTION_MASKS2 = tokens_test['attention_mask']


print(f'INPUT_IDS shape: {INPUT_test.shape}, ATTENTION_MASKS shape: {ATTENTION_MASKS2.shape}')

In [ ]:
X_test = sq.pad_sequences(INPUT_test, maxlen=max_len)
X_test_attention = sq.pad_sequences(ATTENTION_MASKS2, maxlen=max_len)


In [ ]:
test

In [ ]:
y_predict = model.predict([X_test,X_test_attention])
y_predict

In [ ]:
winner_df = pd.DataFrame(y_predict, columns=['winner_model_a', 'winner_model_b', 'winner_tie'])
result_df = pd.concat([test['id'], winner_df], axis=1)


In [ ]:
result_df.to_csv('submission.csv', index=False)
result_df

____________________

In [ ]:
# # Import necessary libraries
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import log_loss
# from sklearn.preprocessing import LabelEncoder

# # Load the data
# train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
# test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# # Inspect the data
# print(train.head())
# print(test.head())

# # Data Preprocessing
# # Combine responses into one text feature
# train['response_combined'] = train['response_a'] + ' ' + train['response_b']
# test['response_combined'] = test['response_a'] + ' ' + test['response_b']

# # Encode the target labels
# label_encoder = LabelEncoder()
# train['winner'] = label_encoder.fit_transform(train[['winner_model_a', 'winner_model_b', 'winner_tie']].idxmax(axis=1))

# # Feature Engineering
# # Vectorize the combined responses using TF-IDF
# tfidf = TfidfVectorizer(max_features=1000)
# X_train_tfidf = tfidf.fit_transform(train['response_combined'])
# X_test_tfidf = tfidf.transform(test['response_combined'])

# # Prepare the data for model training
# X_train, X_val, y_train, y_val = train_test_split(X_train_tfidf, train['winner'], test_size=0.2, random_state=42)

# # Train the model
# model = RandomForestClassifier(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)

# # Validate the model
# y_val_pred_proba = model.predict_proba(X_val)
# val_log_loss = log_loss(y_val, y_val_pred_proba)
# print(f'Validation Log Loss: {val_log_loss}')

# # Predict on the test set
# test_pred_proba = model.predict_proba(X_test_tfidf)

# # Prepare the submission file
# submission = pd.DataFrame(test['id'], columns=['id'])
# submission['winner_model_a'] = test_pred_proba[:, label_encoder.transform(['winner_model_a'])]
# submission['winner_model_b'] = test_pred_proba[:, label_encoder.transform(['winner_model_b'])]
# submission['winner_tie'] = test_pred_proba[:, label_encoder.transform(['winner_tie'])]
# submission.to_csv('submission.csv', index=False)

# # Inspect the submission file
# print(submission.head())